DS 266 Final Project

# Set up
Install and import libraries/dependencies

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# Install HuggingFace transformers
%pip install transformers

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Checking for Rust toolchain....
      
      Cargo, the Rust package manager, is not installed or is not on PATH.
      This package requires Rust and Cargo to compile extensions. Install it through
      the system's package manager or via https://rustup.rs/
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [13]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
%pip install rouge_score

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=5821c86a7dd5a4dcae3874a8e7086f11bfd3ca6a2a62fa813d9cb06c797c59ee
  Stored in directory: c:\users\005\appdata\local\pip\cache\wheels\5f\dd\89\461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Import BART
from transformers import BartForConditionalGeneration, BartTokenizer

In [2]:
import pandas as pd
import numpy as np
from transformers import set_seed
from transformers import Trainer
from transformers import TrainingArguments
import random
from sklearn.model_selection import train_test_split
import evaluate
import re
from datasets import Dataset

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA GeForce RTX 3070 Laptop GPU


## Set Random Seed
One may comment these out for random behaviors

In [4]:
# Set seeds for reproducibility
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)
set_seed(seed_value)

# Datasets

In [5]:
# Datasets file paths, may change per envrionments like local vs Google Colab
# ROCStoriesSpring2016_FILE_PATH = './drive/MyDrive/DS 266/Final/ROCStoriesSpring2016.csv'
# ROCStoriesWinter2017_FILE_PATH = './drive/MyDrive/DS 266/Final/ROCStoriesWinter2017.csv'
# ClozeTestWinter2018ValData_FILE_PATH = './drive/MyDrive/DS 266/Final/ClozeTestWinter2018ValData.csv'
# ClozeTestWinter2018TestData_FILE_PATH = './drive/MyDrive/DS 266/Final/ClozeTestWinter2018TestData.csv'
ROCStoriesSpring2016_FILE_PATH = './ROCStoriesSpring2016.csv'
ROCStoriesWinter2017_FILE_PATH = './ROCStoriesWinter2017.csv'
ClozeTestWinter2018ValData_FILE_PATH = './ClozeTestWinter2018ValData.csv'
ClozeTestWinter2018TestData_FILE_PATH = './ClozeTestWinter2018TestData.csv'

In [6]:
# Load data
ROCStoriesSpring2016 = pd.read_csv(ROCStoriesSpring2016_FILE_PATH)
ROCStoriesWinter2017 = pd.read_csv(ROCStoriesWinter2017_FILE_PATH)
ClozeTestWinter2018ValData = pd.read_csv(ClozeTestWinter2018ValData_FILE_PATH)
ClozeTestWinter2018TestData = pd.read_csv(ClozeTestWinter2018TestData_FILE_PATH)

## Inspect Datasets
Check shapes and duplicates

In [7]:
print(ROCStoriesSpring2016.shape)
ROCStoriesSpring2016.head()

(45496, 7)


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a c...,They got themselves and Dan on a diet.
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a w...,The bike frame bent and Carrie got a deep gash...
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long...,"After walking for over a mile, something happe...",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to h...
3,d173b7de-4611-4cdf-934c-912834755e41,The bad customer.,Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",He began yelling about how long his food was t...,Jane didn't know how to react.,"Luckily, her coworker intervened and calmed th..."
4,af0fd5a4-de36-47ba-8aa2-e99d10986d7a,Being Patient,I was talking to my crush today.,She continued to complain about guys flirting ...,I decided to agree with what she says and list...,"After I got home, I got a text from her.",She asked if we can hang out tomorrow.


In [8]:
print(ROCStoriesWinter2017.shape)
ROCStoriesWinter2017.head()

(52665, 7)


,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.


In [9]:
print(ClozeTestWinter2018ValData.shape)
ClozeTestWinter2018ValData.head()

(1571, 8)


,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1
1,bff9f820-9605-4875-b9af-fe6f14d04256,Laverne needs to prepare something for her fri...,She decides to bake a batch of brownies.,She chooses a recipe and follows it closely.,Laverne tests one of the brownies to make sure...,The brownies are so delicious Laverne eats two...,Laverne doesn't go to her friend's party.,1
2,e8f628d5-9f97-40ed-8611-fc0e774673c4,Sarah had been dreaming of visiting Europe for...,She had finally saved enough for the trip.,She landed in Spain and traveled east across t...,She didn't like how different everything was.,Sarah then decided to move to Europe.,Sarah decided that she preferred her home over...,2
3,f5226bfe-9f26-4377-b05f-3d9568dbdec1,Gina was worried the cookie dough in the tube ...,She was very happy to find she was wrong.,The cookies from the tube were as good as from...,Gina intended to only eat 2 cookies and save t...,Gina liked the cookies so much she ate them al...,Gina gave the cookies away at her church.,1
4,69ac9b05-b956-402f-9fff-1f926ef9176b,It was my final performance in marching band.,I was playing the snare drum in the band.,We played Thriller and Radar Love.,The performance was flawless.,I was very proud of my performance.,I was very ashamed of my performance.,1


In [10]:
print(ClozeTestWinter2018TestData.shape)
ClozeTestWinter2018TestData.head()

(1571, 7)


,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2
0,f6aad64a-e34c-415d-b895-dbfa187ed43e,Bob was bored at his job as a school teacher.,He had been working so hard this past month.,He decided to treat himself with something spe...,He ordered tickets for a weekend snowboarding ...,He was looking forward to getting away.,His boss told him he had to work this weekend.
1,0fedd90d-5295-4b79-b2d0-15a2bad624ee,Olivia went out with Harry on a date.,Harry thought the date was going well.,Olivia thinks he is a complete jerk and never ...,Harry keeps raving about their chemistry.,Olivia is about to leave.,"Olivia had her friend call her, to tell her th..."
2,018152fd-f984-4d05-ad1e-12f1fb7eceb6,Jack and Ferris always fought for headphones.,One day Jack broke Ferris' headphones while jo...,Ferris was furious at Jack.,Their parents yelled at them.,Jack promised Ferris to buy him new headphones.,Jack promised to take Ferris jogging.
3,feef76df-b75a-4501-9c1a-f8a7b6ee442f,I needed someone to help me move a bed across ...,"I called a couple of friends, but they were busy.","Finally, I called my grandson, who came right ...",He helped me move the bed to the right spot.,I made him an ice cream sundae for his efforts.,"He was tired from moving it, so he took a nap ..."
4,929eaf8b-a175-4460-a885-43be8a89ca62,Hannah had a beautiful cat that she loved very...,"However, she noticed that her cat was getting ...","One day, her cat ran away and never came back.",Hannah was devastated.,She never saw her beautiful cat again.,"The next day, she saw the cat walking down the..."


Please note that ClozeTestWinter2018TestData has no ground truth provided for us. We have to ask ROCStories and Story Cloze Test owner for grading. Therefore, for our final project evaluation, we may need to have another test dataset.

In [11]:
pd.merge(ROCStoriesSpring2016, ROCStoriesWinter2017, how='inner').empty

True

In [12]:
# In case some rows have different story ids but the same sentences
pd.merge(ROCStoriesSpring2016, ROCStoriesWinter2017, how='inner', on=["sentence1", "sentence2"]).empty

True

We can see that two datasets have no duplicate. We can safely combine them

In [13]:
ROCStories = pd.concat([ROCStoriesSpring2016, ROCStoriesWinter2017])
ROCStories

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a c...,They got themselves and Dan on a diet.
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a w...,The bike frame bent and Carrie got a deep gash...
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long...,"After walking for over a mile, something happe...",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to h...
3,d173b7de-4611-4cdf-934c-912834755e41,The bad customer.,Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",He began yelling about how long his food was t...,Jane didn't know how to react.,"Luckily, her coworker intervened and calmed th..."
4,af0fd5a4-de36-47ba-8aa2-e99d10986d7a,Being Patient,I was talking to my crush today.,She continued to complain about guys flirting ...,I decided to agree with what she says and list...,"After I got home, I got a text from her.",She asked if we can hang out tomorrow.
...,...,...,...,...,...,...,...
52660,134e8636-3617-43d8-ba6a-9a11b3b115b1,Flavor,The man liked the flavor.,He tried to recreate it at home.,He could not get the flavor right.,He asked the owner of the recipe for help.,The owner of the flavor sold him the recipe.
52661,4c317f76-ca42-4024-a4c2-12ec911cf89b,After Death,"After my friend's dad's funeral, I got in trou...",The principal said I wasn't allowed to leave s...,He found out I had my friend sign me out.,He told me I was getting detention.,I skipped detention all week.
52662,a18fd0d2-4d0c-4316-befe-e3d827fe699b,Janice breaks her wrist,Janice was out exercising for her big soccer g...,She was doing some drills with her legs.,While working out and exercising she slips on ...,She falls down and uses her wrist to break her...,She breaks her wrist in the process and goes t...
52663,2c14252b-4080-4fca-8765-537772018508,Jamie marries for love,Jamie is an american girl.,Jamie wants to get married to a mexican man.,Her family assumes it's because the man wants ...,Jamie insist that she is marrying him out of l...,Jamie gets married and they spent the rest of ...


# Utility Functions

In [14]:
def combine_sentences(row, sentence_columns_to_be_combined=["sentence1", "sentence2", "sentence3", "sentence4"]):
  assert len(sentence_columns_to_be_combined) > 0, "Sentence columns to be combined list length has to be larger than 0"
  combined_sentence = row[sentence_columns_to_be_combined[0]]
  for i in range(1, len(sentence_columns_to_be_combined)):
    combined_sentence += ' ' + row[sentence_columns_to_be_combined[i]]
  return combined_sentence

In [15]:
def remove_first_four_sentences(paragraph):
    # Split the paragraph into sentences using regex
    sentences = re.split(r'(?<=[.!?])[\s"]+', paragraph)
    # Keep all sentences except the first 4
    remaining_sentences = sentences[4:]
    # Rejoin the sentences into a paragraph
    joined_sentences = ""
    for sentence in remaining_sentences:
      joined_sentences += sentence
      joined_sentences += " "
    joined_sentences = joined_sentences[:-1]
    return joined_sentences

def vectorized_remove_first_four_sentences(paragraphs):
    return np.vectorize(remove_first_four_sentences)(np.array(paragraphs))

# Train, Validation, Test Split

In [16]:
ROCStories_Y = ROCStories["sentence5"]
ROCStories_X = ROCStories.drop(columns=["storyid", "storytitle", "sentence5"])
ROCStories_X = ROCStories_X.apply(combine_sentences, axis=1)
ROCStories_X

0        Dan's parents were overweight. Dan was overwei...
1        Carrie had just learned how to ride a bike. Sh...
2        Morgan enjoyed long walks on the beach. She an...
3        Jane was working at a diner. Suddenly, a custo...
4        I was talking to my crush today. She continued...
                               ...                        
52660    The man liked the flavor. He tried to recreate...
52661    After my friend's dad's funeral, I got in trou...
52662    Janice was out exercising for her big soccer g...
52663    Jamie is an american girl. Jamie wants to get ...
52664    The orange fell from the tree. It hit a girl o...
Length: 98161, dtype: object

In [17]:
ROCStories_X.shape

(98161,)

In [18]:
ROCStories_Y.shape

(98161,)

We decided to combine some of ROCStories data and the Story Cloze 2018 Validation datasets as the final train datasets. The reasoning to additionally include Story Cloze dataset is that Story Cloze 2018 Validation datasets have the correct endings, therefore can be used for both classification or story ending generation evaluation. It also has a potential to be a bit different from the original ROCStories datasets that is mainly used as training data (in other words, increasing the variance of the test datasets for generalization).

In [19]:
ROCX_train_val, ROCX_test, ROCy_train_val, ROCy_test = train_test_split(ROCStories_X, ROCStories_Y, test_size=0.02, random_state=seed_value)

In [20]:
print(ROCX_train_val.shape)
print(ROCX_test.shape)
print(ROCy_train_val.shape)
print(ROCy_test.shape)

(96197,)
(1964,)
(96197,)
(1964,)


In [21]:
Clozey_test = np.where(ClozeTestWinter2018ValData["AnswerRightEnding"]==1, ClozeTestWinter2018ValData["RandomFifthSentenceQuiz1"], ClozeTestWinter2018ValData["RandomFifthSentenceQuiz2"])
Clozey_test = pd.DataFrame(Clozey_test, columns=["sentence5"])
Clozey_test

,sentence5
0,He is happy now.
1,The brownies are so delicious Laverne eats two...
2,Sarah decided that she preferred her home over...
3,Gina liked the cookies so much she ate them al...
4,I was very proud of my performance.
...,...
1566,I have very fond memories of checkers.
1567,She loved her new phone.
1568,They were on sale.
1569,She was offered the new job at a higher salary.


In [22]:
ClozeX_test = ClozeTestWinter2018ValData.drop(columns=["InputStoryid", "RandomFifthSentenceQuiz1", "RandomFifthSentenceQuiz2", "AnswerRightEnding"])
ClozeX_test = ClozeX_test.rename(columns={"InputSentence1": "sentence1", "InputSentence2": "sentence2", "InputSentence3": "sentence3", "InputSentence4": "sentence4"})
ClozeX_test = ClozeX_test.apply(combine_sentences, axis=1)
ClozeX_test

0       Rick grew up in a troubled household. He never...
1       Laverne needs to prepare something for her fri...
2       Sarah had been dreaming of visiting Europe for...
3       Gina was worried the cookie dough in the tube ...
4       It was  my final performance in marching band....
                              ...                        
1566    When I was a kid I really wanted to play check...
1567    Ivy wanted a cell phone like all her friends. ...
1568    Dave walked into the grocery store. He was goi...
1569    Ramona was very unhappy in her job. She asked ...
1570    I wanted to buy a video game console. I asked ...
Length: 1571, dtype: object

In [23]:
X_test = pd.concat([ROCX_test, ClozeX_test])
X_test

20391    Evan had been saving for years. He went to the...
32389    Serena was planning a surprise for her husband...
8700     Fred slapped another man's butt. He thought he...
45048    I used to lived in Phoenix Arizona. On my way ...
23146    Tom thought he was really strong. He challenge...
                               ...                        
1566     When I was a kid I really wanted to play check...
1567     Ivy wanted a cell phone like all her friends. ...
1568     Dave walked into the grocery store. He was goi...
1569     Ramona was very unhappy in her job. She asked ...
1570     I wanted to buy a video game console. I asked ...
Length: 3535, dtype: object

In [24]:
y_test = pd.concat([ROCy_test, Clozey_test])
y_test

,sentence5
20391,Evan knew he looked cool in the new car.
32389,Together they were able to surprise him perfec...
8700,Fred apologized to the man.
45048,Talking with him not only made my day it made ...
23146,Tom's friends thought it was annoying.
...,...
1566,I have very fond memories of checkers.
1567,She loved her new phone.
1568,They were on sale.
1569,She was offered the new job at a higher salary.


In [25]:
ROCX_train, ROCX_val, ROCy_train, ROCy_val = train_test_split(ROCX_train_val, ROCy_train_val, test_size=0.2, random_state=seed_value)

In [26]:
ROCX_train = ROCX_train.to_list()
ROCX_val = ROCX_val.to_list()
ROCy_train = ROCy_train.to_list()
ROCy_val = ROCy_val.to_list()
X_test = X_test.to_list()
y_test = y_test["sentence5"].to_list()

In [27]:
print(len(ROCX_train))
print(len(ROCX_val))
print(len(ROCy_train))
print(len(ROCy_val))
print(len(X_test))
print(len(y_test))

76957
19240
76957
19240
3535
3535


# Bart Model and Baseline

In [55]:
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [56]:
# Tokenize the input text
inputs = tokenizer(X_test, padding=True, return_tensors="pt")
inputs

{'input_ids': tensor([[    0,   717,  9965,  ...,     1,     1,     1],
        [    0,   104,  2816,  ...,     1,     1,     1],
        [    0, 33153, 18361,  ...,     1,     1,     1],
        ...,
        [    0, 33857,  3203,  ...,     1,     1,     1],
        [    0, 32361,  4488,  ...,     1,     1,     1],
        [    0,   100,   770,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

TODO: It's very consuming to do generate. So let's use only a small subset of test data for now.

In [57]:
%%time
# Generate the story ending
with torch.no_grad():
  generated_ids = model.generate(**inputs, min_new_tokens=70) #Set min_new_tokens to ensure an ending is generated
generated_ids

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2842931840 bytes.

In [ ]:
# Decode the generated ending
# Please note that currently generated text includes both the input, story beginning, and the generated ending
full_story_with_ending_test = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
full_story_with_ending_test[0]

"Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. It was a big hit with his friends and family. He was so excited to have a new car to drive around in. It's been a big success. He's so proud."

: 

In [ ]:
perplexity = evaluate.load("perplexity", module_type="metric")
perplexity_results = perplexity.compute(predictions=full_story_with_ending_test, model_id='facebook/bart-large-cnn')

In [ ]:
perplexity_results["mean_perplexity"]

188189.63388671874

Let's also see the original story's perplexity score

In [ ]:
original_story_with_endings_test = [f"{p1} {p2}" for p1, p2 in zip(X_test, y_test)]
original_story_with_endings_test[0]

'Evan had been saving for years. He went to the dealership and bought a really fancy BMW. Evan was so proud of his new car. He showed it off around town. Evan knew he looked cool in the new car.'

TODO: For comparison, this sets to be a subset of test dataset

In [ ]:
perplexity.compute(predictions=original_story_with_endings_test, model_id='facebook/bart-large-cnn')["mean_perplexity"]

  0%|          | 0/4 [00:00<?, ?it/s]

626820.758828125

Now also see Rouge. We use Rouge our problem is more like a summarization, more abstractive and open ended; therefore, when comparing to the ereference, we care more about how much reference info has been captured rather than if the whole generated text is strictly alike the reference.

In [ ]:
empty_indices = [i for i, s in enumerate(vectorized_remove_first_four_sentences(full_story_with_ending_test)) if s == ""]
empty_indices

[20]

In [ ]:
X_test[20]

'Bella came to school with an unexpected present on her desk. To her surprise, she was given a bouquet of beautiful flowers. Bella took a look at the note and it said "Secret Admirer". She took a sniff of the flowers as they smelled wonderful.'

In [ ]:
full_story_with_ending_test[20]

'Bella came to school with an unexpected present on her desk. To her surprise, she was given a bouquet of beautiful flowers. Bella took a look at the note and it said "Secret Admirer" She took a sniff of the flowers as they smelled wonderful. They were from her secret admirer, who she has never met.'

In [ ]:
full_story_with_ending_test[20] = 'Bella came to school with an unexpected present on her desk. To her surprise, she was given a bouquet of beautiful flowers. Bella took a look at the note and it said "Secret Admirer". She took a sniff of the flowers as they smelled wonderful. They were from her secret admirer, who she has never met.'

In [ ]:
empty_indices = [i for i, s in enumerate(vectorized_remove_first_four_sentences(full_story_with_ending_test)) if s == ""]
empty_indices

[]

In [ ]:
rouge = evaluate.load('rouge')
rouge_results = rouge.compute(predictions=vectorized_remove_first_four_sentences(full_story_with_ending_test), references=y_test)

In [ ]:
rouge_results

{'rouge1': 0.15362326568947524,
 'rouge2': 0.02371271210419737,
 'rougeL': 0.12107908619351965,
 'rougeLsum': 0.12140986918418589}

# Bart Model Fine Tuning

In [43]:
train_encodings = tokenizer(ROCX_train, padding=True, max_length=512)
val_encodings = tokenizer(ROCX_val, padding=True, max_length=512)

c:\Users\005\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2847: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [44]:
train_labels = tokenizer(ROCy_train, padding=True, max_length=128, return_tensors="pt")['input_ids']
val_labels = tokenizer(ROCy_val, padding=True, max_length=128, return_tensors="pt")['input_ids']

In [45]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

In [46]:
training_args = TrainingArguments(
    report_to="none",
    output_dir='./results',          # output directory for model predictions and checkpoints
    evaluation_strategy="epoch",     # Evaluation frequency
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size per GPU for training
    per_device_eval_batch_size=16,   # Batch size per GPU for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
)

c:\Users\005\anaconda3\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [48]:
trainer.train()

  0%|          | 0/28860 [00:00<?, ?it/s]

KeyboardInterrupt: 